In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import pandas as pd
import plotly
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [2]:
human = pd.read_csv('human_related_rank.csv')

In [3]:
human

,Unnamed: 0,City,State,Population,Violent,Rape,Robbery,Colleges,Percent_graduate_degree,AvgSATScore,...,NumHospital,total crime,crime_rank,hospital_rank,early_education_rank,old_education_rank,Human_related_rank,latitude,longitude,text
0,0,Atlanta,GA,410606.0,1119.6,36.6,429.3,12.0,12.7,1208.0,...,113.0,1585.5,33.0,22.0,29.0,16.5,27.0,33.749099,-84.390185,Atlanta: 27
1,1,Austin,TX,797215.0,372.5,51.9,99.0,9.0,11.0,1221.0,...,378.0,523.4,3.0,2.5,22.5,14.0,5.0,30.271129,-97.743700,Austin: 5
2,2,Baltimore,MD,612701.0,1535.9,46.2,694.2,11.0,8.5,1227.0,...,52.0,2276.3,40.0,36.0,20.0,20.0,36.0,39.290882,-76.610759,Baltimore: 36
3,3,Birmingham,AL,212157.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,45.0,38.0,35.0,36.0,48.0,33.520682,-86.802433,Birmingham: 48
4,4,Boston,MA,615462.0,706.8,36.1,233.1,22.0,13.8,1277.0,...,77.0,976.0,18.0,30.0,4.0,9.0,10.0,42.360482,-71.059568,Boston: 10
5,5,Buffalo,NY,NaN,1118.6,67.0,400.2,NaN,NaN,1234.0,...,195.0,1585.8,34.0,14.5,16.5,36.0,28.0,42.886717,-78.878392,Buffalo: 28
6,6,Charlotte,NC,745596.0,677.6,24.6,221.8,8.0,8.6,NaN,...,NaN,924.0,16.0,38.0,35.0,23.5,33.5,35.227087,-80.843127,Charlotte: 33
7,7,Chicago,IL,2679998.0,903.8,52.5,353.6,42.0,9.3,1241.0,...,143.0,1309.9,25.0,21.0,11.0,2.5,8.0,41.875555,-87.624421,Chicago: 8
8,8,Cincinnati,OH,292050.0,925.0,79.1,423.1,8.0,8.9,1240.0,...,151.0,1427.2,28.0,19.0,13.0,29.0,21.0,39.101454,-84.512460,Cincinnati: 21
9,9,Cleveland,OH,396441.0,1334.3,124.0,769.3,8.0,4.5,1240.0,...,151.0,2227.6,39.0,19.0,13.0,31.0,29.5,41.505161,-81.693445,Cleveland: 29


In [20]:
app = dash.Dash()

available_indicators = ['crime_rank', 'hospital_rank','early_education_rank', 'old_education_rank']
app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='First-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'crime_rank'
            )
            ],
        style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.Dropdown(
                id='Second-care',
                options=[{'label': i, 'value': i} for i in available_indicators],
                #value = 'hospital_rank'
            )
            ],
        style={'width': '48%', 'display': 'inline-block'}),
        
        html.Button('Submit', id='button'),
        
        html.Div(dcc.Graph(id='indicator-graphic'))
    ]),
    #dcc.Graph(id='indicator-graphic')
])

@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('button', 'n_clicks')],
    [dash.dependencies.State('First-care', 'value'),
     dash.dependencies.State('Second-care', 'value')])
    
def update_graph(Button, First_care, Second_care):
    df = pd.DataFrame()
    df['City'] = human['City']
    df['First'] = human[First_care]
    df['Second'] = human[Second_care]
    df['Total'] = (df['First']+df['Second']).rank(ascending=1)
    df['reverse_rank'] = df['Total'].rank(ascending=0)
    df['text'] = df['City'] + '<br># Final Rank ' + (df['Total']).astype(str)
    df['longitude'] = human['longitude']
    df['latitude'] = human['latitude']
    
    limits = [(0,10),(10,20),(20,30),(30,40),(40,50)]
    colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
    cities = []


    for i in range(len(limits)):
        lim = limits[i]
        df_sub = df[lim[0]:lim[1]]
        city = dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df_sub['longitude'],
            lat = df_sub['latitude'],
            text = df_sub['text'],
            marker = dict(
                size = df_sub['reverse_rank']*15,
                color = colors[i],
                line = dict(width=0.5, color='rgb(40,40,40)'),
                sizemode = 'area'
            ),
            name = '{0} - {1}'.format(lim[0],lim[1]) )
        cities.append(city)

        layout = dict(
            title = 'The human related ranking of US big cities',
            showlegend = True,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(217, 217, 217)',
                subunitwidth=1,
                countrywidth=1,
                subunitcolor="rgb(255, 255, 255)",
                countrycolor="rgb(255, 255, 255)"
            ),
        )
        fig = dict( data=cities, layout=layout )
        
    return {   
        plotly.offline.plot( fig, validate=False )     
    }

app.run_server()

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Nov/2017 21:40:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2017 21:40:21] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2017 21:40:21] "GET /_dash-dependencies HTTP/1.1" 200 -
[2017-11-17 21:40:21,541] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\zhang\Documents\anaconda\lib\site-packages\flask\app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\zhang\Documents\anaconda\lib\site-packages\flask\app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\zhang\Documents\anaconda\lib\site-packages\flask\app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\zhang\Documents\anaconda\lib\site-packages\flask\_compat.py", line 33, in reraise
    raise value
  File "C:\Users\zhang\Documents\anaconda\lib\site-packa